## Marvin
> "Let’s build robots with Genuine People Personalities," they said. So they tried it out with me. I’m a personality prototype. You can tell, can’t you?

## Note:
- Marvin is purely for internal use
- Marvin is unrelated to the Prefect slack app that customers will interact with

## What we want to talk about today:
- what makes Marvin tick?
- what do you need locally to hack on Marvin?

<img src="marvins_guts.svg">

#### Marvin's high-level structure


```
|____marvin
| |____defcon.py      # /defcon command
| |____responses.py   # notifications / reactions
| |______init__.py
| |____core.py        # webserver
| |____loop_policy.py # utility for scheduling standup
| |____standup.py     # standup logic
| |____utilities.py   # slack utilities

## deployment
|____kubernetes
| |____deployment.yml
| |____service.yml

|____.circleci
| |____config.yml
```

## And unfortunately, there's a relatively large portion of work that needs to occur in the Slack Web UI
<img src="slack_web_ui.png">

# Tokens

| Token Name        | Purpose           |
| ------------- |:-------------:| -----:|
| `MARVIN_TOKEN`      | Marvin's "bot user" token; this is the token Marvin needs for simply speaking / reacting |
| `MARVIN_OAUTH_TOKEN`      | Marvin's full OAuth token; this is the token Marvin needs for performing higher permission actions (such as editing pins) |
| `SLACK_VALIDATION_TOKEN` | this token is used to internally validate that incoming requests are legitimate Slack requests |
| `GOOGLE_SERVICE_ACCOUNT_KEY` | Marvin's key granting permissions for Googe Cloud services |

# Things you can do locally with _only_ these tokens:

- Speak through Marvin:

```python
from marvin.utilities import say

say("Hey this is really Chris", channel="CBH18KG8G") # engineering channel
```

- edit Marvin's code, run tests (of course there are unit tests!)
- deploy Marvin from your laptop
- anything that doesn't require a running webserver

# Marvin's webserver

Slash commands and event subscriptions are sent to a fixed web address, which normally points to a Google Cloud IP Address; you can easily change this to point to a local (and temporary) address.  Run the `marvin` CLI command + `ngrok http 8080` to standup a webserver and
<img src="ngrok_slack.png" width=500>

Notice the error message: this is the `SLACK_VALIDATION_TOKEN`

- webserver is useful for capturing things and inspecting what slack is sending, although the API docs aren't bad
- how notifications work (marvin is just in the channel, show code snippet which redirects for github and notion)
- what you can do (marvin needs a memory, marvin needs more fun - he should emoji react to stuff)
- get creative! (meme)

# The Future: Marvin will run on Prefect!